In [1]:
#@title Instalar Spark
# Instalar SDK java 8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Descargar Spark 3.4.3
!wget -q https://archive.apache.org/dist/spark/spark-3.4.3/spark-3.4.3-bin-hadoop3.tgz

# Descomprimir el archivo descargado de Spark
!tar xf spark-3.4.3-bin-hadoop3.tgz

# Establecer las variables de entorno
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.4.3-bin-hadoop3"

# Instalar la librería findspark
!pip install -q findspark

# Instalar pyspark
!pip install -q pyspark


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [2]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

In [3]:
# Crear un DataFrame mediante la lectura de un archivo de texto
df = spark.read.text('./data/dataTXT.txt')

In [4]:
# Mostrar dataframe
df.show()

+--------------------+
|               value|
+--------------------+
|Estamos en el cur...|
|En este capítulo ...|
|En esta sección e...|
|y en este ejemplo...|
+--------------------+



In [5]:
# Mostrar dataframe sin truncar los campos
df.show(truncate=False)

+-----------------------------------------------------------------------+
|value                                                                  |
+-----------------------------------------------------------------------+
|Estamos en el curso de pyspark                                         |
|En este capítulo estamos estudiando el API SQL de Saprk                |
|En esta sección estamos creado dataframes a partir de fuentes de datos,|
|y en este ejemplo creamos un dataframe a partir de un texto plano      |
+-----------------------------------------------------------------------+



In [7]:
# Crear un DataFrame mediante la lectura de un archivo csv
df1 = spark.read.csv('./data/dataCSV.csv')
df1.show()


AnalysisException: [PATH_NOT_FOUND] Path does not exist: file:/content/data/dataCSV.csv.

In [ ]:
df1 = spark.read.option('header', 'true').csv('./data/dataCSV.csv')
df1.show()

In [9]:
# Leer un archivo de texto con un delimitador diferente
df2 = spark.read.option('header', 'true').option('delimiter', '|').csv('./data/dataTab.txt')
df2.show()


+----+----+----------+-----+
|pais|edad|     fecha|color|
+----+----+----------+-----+
|  MX|  23|2021-02-21| rojo|
|  CA|  56|2021-06-10| azul|
|  US|  32|2020-06-02|verde|
+----+----+----------+-----+



In [10]:
# Crear un DataFrame a partir de un json proporcionando un schema
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DateType

In [11]:
json_schema =  StructType(
    [
     StructField('color', StringType(), True),
     StructField('edad', IntegerType(), True),
     StructField('fecha', DateType(), True),
     StructField('pais', StringType(), True)
    ]
)

In [12]:
df4 = spark.read.schema(json_schema).json('./data/dataJSON.json')
df4.show()

+-----+----+----------+----+
|color|edad|     fecha|pais|
+-----+----+----------+----+
| rojo|null|2021-02-21|  MX|
| azul|null|2021-06-10|  CA|
|verde|null|2020-06-02|  US|
+-----+----+----------+----+



In [13]:
df4.printSchema()


root
 |-- color: string (nullable = true)
 |-- edad: integer (nullable = true)
 |-- fecha: date (nullable = true)
 |-- pais: string (nullable = true)



In [14]:
# Crear un DataFrame a partir de un archivo parquet
df5 = spark.read.parquet('./data/dataPARQUET.parquet')
df5.show()

Py4JJavaError: An error occurred while calling o90.parquet.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 9.0 failed 1 times, most recent failure: Lost task 0.0 in stage 9.0 (TID 9) (3d6460ad3ef9 executor driver): org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:322)
	at org.apache.spark.util.ThreadUtils$.parmap(ThreadUtils.scala:396)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetFileFormat$.readParquetFootersInParallel(ParquetFileFormat.scala:422)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetFileFormat$.$anonfun$mergeSchemasInParallel$1(ParquetFileFormat.scala:472)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetFileFormat$.$anonfun$mergeSchemasInParallel$1$adapted(ParquetFileFormat.scala:464)
	at org.apache.spark.sql.execution.datasources.SchemaMergeUtils$.$anonfun$mergeSchemasInParallel$2(SchemaMergeUtils.scala:7

In [ ]:
# Otra alternativa para leer desde una fuente de datos parquet en este caso
df6 = spark.read.format('parquet').load('./data/dataPARQUET.parquet')
df6.printSchema()

In [ ]:
"""spark = SparkSession.builder.getOrCreate()

# Crear un DataFrame mediante la lectura de un archivo de texto

df = spark.read.text('./data/dataTXT.txt')

df.show()

df.show(truncate=False)

# Crear un DataFrame mediante la lectura de un archivo csv

df1 = spark.read.csv('./data/dataCSV.csv')

df1.show()

df1 = spark.read.option('header', 'true').csv('./data/dataCSV.csv')

df1.show()

# Leer un archivo de texto con un delimitador diferente

df2 = spark.read.option('header', 'true').option('delimiter', '|').csv('./data/dataTab.txt')

df2.show()

# Crear un DataFrame a partir de un json proporcionando un schema

from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DateType

json_schema =  StructType(
    [
     StructField('color', StringType(), True),
     StructField('edad', IntegerType(), True),
     StructField('fecha', DateType(), True),
     StructField('pais', StringType(), True)
    ]
)

df4 = spark.read.schema(json_schema).json('./data/dataJSON.json')

df4.show()

df4.printSchema()

# Crear un DataFrame a partir de un archivo parquet

df5 = spark.read.parquet('./data/dataPARQUET.parquet')

df5.show()

# Otra alternativa para leer desde una fuente de datos parquet en este caso

df6 = spark.read.format('parquet').load('./data/dataPARQUET.parquet')

df6.printSchema()"""